# Preprocesamiento de Datos para uso API

## Objetivo
Para optimizar el rendimiento del API, creamos dataframes intermedios con la minima data requerida por la funciones relacionadas. De esta forma, consumimos menos recursos y agilizamos el response-time para el usuario del API.

In [1]:
# Importación de librerías
import os
import pickle
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity

### Carga de data procesados en 'ETL.ipynb'

In [9]:
nombre_dir = 'data_pickle'

# Dataframes de steam_games
df_steamgames: pd.DataFrame = pd.read_pickle(f'./{nombre_dir}/df_steamgames_clean.pkl')
df_steamgames_genres: pd.DataFrame = pd.read_pickle(f'./{nombre_dir}/df_steamgames_genres.pkl')
df_steamgames_specs: pd.DataFrame = pd.read_pickle(f'./{nombre_dir}/df_steamgames_specs.pkl')
df_steamgames_tags: pd.DataFrame = pd.read_pickle(f'./{nombre_dir}/df_steamgames_tags.pkl')

# Dataframes de user_reviews
df_userreviews: pd.DataFrame = pd.read_pickle(f'./{nombre_dir}/df_userreviews_clean.pkl')

# Dataframes de users_item
df_usersitems: pd.DataFrame = pd.read_pickle(f'./{nombre_dir}/df_usersitems_clean.pkl')

## Función: PlayTimeGenre

Esta función devuelve año con mas horas jugadas para dicho género.

In [10]:
# Extraemos las columnas requeridas de df_steamgames
df_release_year = df_steamgames[['id', 'release_year']]

# Extraemos las columnas requeridas de df_usersitems
# Agrupando por 'item_id' y sumando sus respectivos valores de 'playtime_forever'
df_playtime_total = df_usersitems.groupby('item_id')['playtime_forever'].sum().reset_index()

# Unimos los dos dataframes previos
df_games_year_playtime = pd.merge(df_release_year, df_playtime_total, left_on='id', right_on='item_id', how='inner')

# Unimos este nuevo dataframe a 'df_steamgames_genres'
df_generos_year_playtime = df_games_year_playtime.merge(df_steamgames_genres, left_on='id', right_index=True)

# Finalmente convertimos los nombres de las columnas a todo minúsculas, para facilitar la comparación del input del usuario del API contra el dataframe
df_generos_year_playtime.columns = df_generos_year_playtime.columns.str.lower()
df_generos_year_playtime.head()

,id,release_year,item_id,playtime_forever,strategy,casual,rpg,utilities,free to play,education,...,adventure,design &amp; illustration,action,simulation,early access,animation &amp; modeling,racing,audio production,video production,massively multiplayer
0,282010,1997-01-01,282010,9319.0,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
1,70,1998-01-01,70,2682852.0,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1640,2006-01-01,1640,27397.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1630,2006-01-01,1630,21111.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2400,2006-01-01,2400,1048156.0,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,False


## Función: UserForGenre

Esta función devuelve el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Requerimos dos dataframes intermedios.

* Dataframe intermedio para delvolver usuario que acumula más horas jugadas para el género

In [11]:
# Extraemos las columnas requeridas de df_usersitems
df_usuario_playtime = df_usersitems[['user_id', 'item_id', 'playtime_forever']]

# Unimos df_usuario_playtime y df_steamgames_genres
df_usuario_playtime_generos = df_usuario_playtime.merge(df_steamgames_genres, left_on='item_id', right_index=True)

# Instanciamos una lista para almacenar los resultados del siguiente loop
usuarios_maxplay_por_genero = []

# Iteramos por cada genero en 'df_steamgames_genres'
for genero in df_steamgames_genres.columns:
    # Filtramos 'df_users_genres_playtime' por genero
    df_genero = df_usuario_playtime_generos[df_usuario_playtime_generos[genero]]

    # Agrupamos por 'user_id' y sumamos 'playtime_forever' de cada usuario
    playtime_por_usuario = df_genero.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Identificamos el usuario con el máximo 'playtime_forever' para el género
    usuario_con_maxplay = playtime_por_usuario.loc[playtime_por_usuario['playtime_forever'].idxmax()]
    
    # Almacenamos el resultado
    usuarios_maxplay_por_genero.append({
        'genero': genero.lower(), # almacenamos como todo-minúsculas, para la comparación del input del usuario contra el dataframe
        'user_id': usuario_con_maxplay['user_id'], 
        'playtime_forever': usuario_con_maxplay['playtime_forever']
    })

# Convertimos la lista de resultados a un dataframe
df_maxplay_por_genero = pd.DataFrame(usuarios_maxplay_por_genero)
df_maxplay_por_genero.head()

,genero,user_id,playtime_forever
0,strategy,shinomegami,1141546.0
1,casual,REBAS_AS_F-T,1224933.0
2,rpg,shinomegami,1060592.0
3,utilities,76561198073642113,207651.0
4,free to play,idonothack,808241.0


* Dataframe intermedio para delvolver una lista de la acumulación de horas jugadas por año.

In [12]:
# Extraemos los usuarios con más horas jugadas por género
usuarios_maxplay_por_genero = df_maxplay_por_genero['user_id']

# Extraemos las columnas requeridas de df_steamgames
df_release_year = df_steamgames[['id', 'release_year']]

# Merge df_release_year and df_user_genre_playtime on the 'id' and 'item_id' columns
# Unimos 'df_release_year' y 'df_usuario_playtime_generos'
merged_df = pd.merge(df_release_year, df_usuario_playtime_generos, left_on='id', right_on='item_id')

# Filtramos los usuarios con mayor cantidad de horas jugadas
user_filtered_df = merged_df[merged_df['user_id'].isin(usuarios_maxplay_por_genero)]

# Aplicamos melt() para unir los géneros en una columna
melted_df = pd.melt(user_filtered_df, id_vars=['release_year', 'playtime_forever', 'user_id'], value_vars=user_filtered_df.columns[4:], var_name='genero', value_name='es_genero')

# Filtramos donde 'es_genero' = True
df_genero_filtrado = melted_df[melted_df['es_genero']]

# Agrupamos para obtener una sumatoria de 'playtime_forever' por genero, por año 
df_play_por_genero_por_anio = df_genero_filtrado.groupby(['release_year', 'genero', 'user_id'])['playtime_forever'].sum().reset_index()
df_play_por_genero_por_anio.head()

,release_year,genero,user_id,playtime_forever
0,1987-01-01,Adventure,idonothack,0.0
1,1987-01-01,Simulation,DownSyndromeKid,0.0
2,1987-01-01,Simulation,idonothack,0.0
3,1988-01-01,Action,DownSyndromeKid,664.0
4,1988-01-01,Action,shinomegami,136.0


* Para agilizar la consulta de la función, hacemos lo siguientes 2 pasos:

In [13]:
# Convertimos 'release_year' a solo año, y lo fijamos como el indice
df_play_por_genero_por_anio['release_year'] = df_play_por_genero_por_anio['release_year'].dt.year
df_play_por_genero_por_anio.set_index('release_year', inplace=True)

# Convertimos los str de 'genero' a minúsculas 
df_play_por_genero_por_anio['genero'] = df_play_por_genero_por_anio['genero'].str.lower()
df_play_por_genero_por_anio.head()

,genero,user_id,playtime_forever
release_year,,,
1987,adventure,idonothack,0.0
1987,simulation,DownSyndromeKid,0.0
1987,simulation,idonothack,0.0
1988,action,DownSyndromeKid,664.0
1988,action,shinomegami,136.0


## Feature Engineering - Sentiment Analysis (user_reviews)

* Para las siguientes 3 funciones del API, requerimos primero aplicar un análisis de sentimiento a las reseñas en 'user_reviews'.

In [14]:
# Instanciamos el analizador de vaderSentiment
analyzer = SentimentIntensityAnalyzer()

In [15]:
# Creamos una función que nos permita clasificar las reseñas en la columna 'reviews' de 'user_reviews'
def clasificar_sentiment_analysis(reseña: str):
    """Aplica vaderSentiment.SentimentIntensityAnalyzer a la reseña de entrada, y devuelve un int
    como clasificación de sentiment: 0 si es negativo, 1 si es neutro, 2 si es positivo.
    """
    # Aplicamos el análisis de sentimiento a 'reseña'
    polarity_scores = analyzer.polarity_scores(reseña)
    polarity_scores_compound = polarity_scores['compound']
    
    # Clasificamos el resultado de 'polarity_scores_compound' en 3 categorías
    # '2' si la reseña es positiva
    if polarity_scores_compound >= 0.05:
        return 2
    # '0' si la reseña es negativa
    elif polarity_scores_compound <= -0.05:
        return 0
    # '1' si la reseña es neutral
    else:
        return 1

In [16]:
# Aplicamos nuestra función 'clasificar_sentiment_analysis' a las reseñas en columna 'review'
# y guardamos los resultados en una nueva columna 'sentiment_analysis'
df_userreviews['sentiment_analysis'] = df_userreviews['review'].apply(clasificar_sentiment_analysis)
df_userreviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,sentiment_analysis
0,,2014-06-24,NaT,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,2
0,,2013-09-08,NaT,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,2
0,,2013-11-29,NaT,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,js41637,2
1,,NaT,NaT,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,evcentric,0
1,,2015-12-04,2015-12-05,370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?""",evcentric,2


* Con el análisis realizado, procedemos con el resto del las funciones del API.

## Función: UsersRecommend

Esta función devuelve los top 3 de juegos más recomendados por usuarios para el año dado.

In [17]:
# Extraemos las columnas requeridas de 'df_steamgames' y 'df_userreviews'
df_appname_year = df_steamgames[['id', 'app_name', 'release_year']]
df_recommend_sentiment = df_userreviews[['item_id', 'recommend', 'sentiment_analysis']]

# Unimos los dos dataframes
df_merge = pd.merge(df_appname_year, df_recommend_sentiment, left_on='id', right_on='item_id')

# Filtramos las filas de no recomendados ('recommend' = True, y 'sentiment_analysis' = 1 o 2) 
df_recommend = df_merge[(df_merge['recommend'] == True) & (df_merge['sentiment_analysis'].isin([1, 2]))]

# Agrupamos por 'release_year' y 'app_name', y contamos la frecuencia de recomendados
df_recommend_agrupado = df_recommend.groupby(['release_year', 'app_name']).size().reset_index(name='cantidad_recommend')

# Agrupamos por año y aplicamos una función lambda para extraer los top 3 recomendados por año
df_maxrecommend_por_anio = df_recommend_agrupado.groupby('release_year').apply(lambda x: x.nlargest(3, 'cantidad_recommend')).reset_index(drop=True)

# Convertimos 'release_year' a tipo datetime year para facilitar su uso en el API
df_maxrecommend_por_anio['release_year'] = df_maxrecommend_por_anio['release_year'].dt.year

# El df_maxrecommend_por_anio ahora contiene los top 3 recomendados juegos recomendados, por año
df_maxrecommend_por_anio.head()

/tmp/ipykernel_7740/4203143459.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_maxrecommend_por_anio = df_recommend_agrupado.groupby('release_year').apply(lambda x: x.nlargest(3, 'cantidad_recommend')).reset_index(drop=True)


,release_year,app_name,cantidad_recommend
0,1989,Sword of the Samurai,1
1,1990,Commander Keen,3
2,1990,LOOM™,1
3,1991,Crystal Caves,1
4,1992,Cosmo's Cosmic Adventure,1


## Función: UsersNotRecommend

Esta función devuelve los top 3 de juegos menos recomendados por usuarios para el año dado.

In [18]:
# Extraemos las columnas requeridas de 'df_steamgames' y 'df_userreviews'
df_appname_year = df_steamgames[['id', 'app_name', 'release_year']]
df_recommend_sentiment = df_userreviews[['item_id', 'recommend', 'sentiment_analysis']]

# Unimos los dos dataframes
df_merge = pd.merge(df_appname_year, df_recommend_sentiment, left_on='id', right_on='item_id')

# Filtramos las filas de no recomendados ('recommend' = False, y 'sentiment_analysis' = 0) 
df_notrecommend = df_merge[(df_merge['recommend'] == False) & (df_merge['sentiment_analysis'].eq(0))]

# Agrupamos por 'release_year' y 'app_name', y contamos la frecuencia de no recomendados
df_notrecommend_agrupado = df_notrecommend.groupby(['release_year', 'app_name']).size().reset_index(name='cantidad_not_recommend')

# Agrupamos por año y aplicamos una función lambda para extraer los top 3 no recomendados 
df_minrecommend_por_anio = df_notrecommend_agrupado.groupby('release_year').apply(lambda x: x.nlargest(3, 'cantidad_not_recommend')).reset_index(drop=True)

# Convertimos 'release_year' a tipo datetime.year para facilitar su uso en el API
df_minrecommend_por_anio['release_year'] = df_minrecommend_por_anio['release_year'].dt.year

# El df_minrecommend_por_anio ahora contiene los top 3 juegos no recomendados, por año
df_minrecommend_por_anio.head()

/tmp/ipykernel_7740/3711030725.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_minrecommend_por_anio = df_notrecommend_agrupado.groupby('release_year').apply(lambda x: x.nlargest(3, 'cantidad_not_recommend')).reset_index(drop=True)


,release_year,app_name,cantidad_not_recommend
0,1990,Commander Keen,1
1,1998,YOU DON'T KNOW JACK HEADRUSH,1
2,1999,Team Fortress Classic,2
3,1999,RollerCoaster Tycoon®: Deluxe,1
4,1999,Sven Co-op,1


## Función: sentiment_analysis

Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [19]:
# Extraemos las columnas requeridas de 'df_steamgames' y 'df_userreviews'
df_release_year = df_steamgames[['id', 'release_year']]
df_sentiment_analysis = df_userreviews[['item_id', 'sentiment_analysis']]

# Unimos los dos dataframes
df_merge = pd.merge(df_release_year, df_sentiment_analysis, left_on='id', right_on='item_id')
df_sentiment_por_anio = df_merge.groupby('release_year')['sentiment_analysis'].value_counts().unstack(fill_value=0)
df_sentiment_por_anio.index = df_sentiment_por_anio.index.year
df_sentiment_por_anio.rename(columns={0:'Negativo', 1:'Neutral', 2:'Positivo'}, inplace=True)
df_sentiment_por_anio.head()

sentiment_analysis,Negativo,Neutral,Positivo
release_year,,,
1989,0,0,1
1990,1,0,4
1991,0,0,1
1992,1,0,2
1993,1,0,4


## Función: recomendacion_juego

Es un sistema de recomendación item-item. Ingresando el id de producto, devuelve una lista con 5 juegos recomendados similares al ingresado.

* Los datos en 'genres', 'tags', y 'specs' son candidatos ideales para los features del sistema de recomendación.

### Feature engineering (recomendacion_juego)

In [20]:
df_steamgames_genres.columns

Index(['Strategy', 'Casual', 'RPG', 'Utilities', 'Free to Play', 'Education',
       'Indie', 'Sports', 'Software Training', 'Photo Editing',
       'Web Publishing', 'Adventure', 'Design &amp; Illustration', 'Action',
       'Simulation', 'Early Access', 'Animation &amp; Modeling', 'Racing',
       'Audio Production', 'Video Production', 'Massively Multiplayer'],
      dtype='object')

In [21]:
df_steamgames_tags.columns

Index(['Choices Matter', 'Gothic', 'Building', '1980s', 'Basketball',
       'Cycling', 'Character Action Game', 'Rhythm', 'Text-Based',
       'Cult Classic',
       ...
       'Alternate History', 'Conversation', 'Gambling', 'Funny',
       'Multiple Endings', 'Time Attack', 'Shooter', 'Inventory Management',
       '2.5D', 'LEGO'],
      dtype='object', length=317)

In [22]:
df_steamgames_specs.columns

Index(['Online Co-op', 'Steam Achievements', 'Online Multi-Player',
       'Steam Leaderboards', 'Multi-player', 'Shared/Split Screen',
       'Includes Source SDK', 'In-App Purchases', 'Partial Controller Support',
       'Cross-Platform Multiplayer', 'Commentary available',
       'Mods (require HL1)', 'Captions available', 'Single-player',
       'Steam Trading Cards', 'Steam Cloud', 'Mods',
       'Valve Anti-Cheat enabled', 'MMO', 'Steam Workshop',
       'Includes level editor', 'Downloadable Content', 'Local Co-op',
       'Mods (require HL2)', 'Local Multi-Player', 'Steam Turn Notifications',
       'Stats', 'SteamVR Collectibles', 'Game demo',
       'Full controller support'],
      dtype='object')

Tenemos 2 observaciones:
* 'df_steamgames_genres' contiene géneros que no aplican a juegos (ej. "Photo Editing"). Estas columnas no serán incluidas como features.
* 'df_steamgames_tags' contiene 317 categorías. No incluiremos 'tags' por cuestión de recursos computacional limitados. También consideramos que la combinación de 'genres' y 'specs' contiene suficiente información para crear un sistema de recomendación adecuado.

In [23]:
# Descartamos las columns de 'df_steamgames_genres' los géneros que no aplican a juegos 
df_steamgames_genres.drop(['Utilities', 'Software Training', 'Photo Editing', 'Web Publishing', 'Design &amp; Illustration',
                           'Animation &amp; Modeling', 'Audio Production', 'Video Production'
                           ],
                           axis=1, inplace=True
)

In [4]:
# Creamos el dataframe de features uniendo 'genres' y 'specs'
df_features = pd.merge(df_steamgames_genres, df_steamgames_specs, left_index=True, right_index=True)
df_features.head()

,Strategy,Casual,RPG,Free to Play,Education,Indie,Sports,Adventure,Action,Simulation,...,Includes level editor,Downloadable Content,Local Co-op,Mods (require HL2),Local Multi-Player,Steam Turn Notifications,Stats,SteamVR Collectibles,Game demo,Full controller support
id,,,,,,,,,,,,,,,,,,,,,
761140,True,True,False,False,False,True,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
643980,True,False,True,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
670290,False,True,False,True,False,True,True,False,False,True,...,False,False,False,False,False,False,True,False,False,False
767400,False,True,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
772540,False,False,False,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [5]:
# Por cuestión de recursos computacionales limitado, utilizaremos los top 500 juegos mas populares para nuestro sistema de recomendación
n = 500
n_juegos_mas_populares = df_usersitems['item_id'].value_counts().head(n).index

# Filtramos 'df_features_filtrados' para solo incluir los top 500 por popularidad
df_features_filtrados = df_features[df_features.index.isin(n_juegos_mas_populares)]

In [6]:
# Aplicamos la similitud de coseno a los top 500 juegos
matriz_similitud_coseno = cosine_similarity(df_features_filtrados.values, df_features_filtrados.values)

# Convertimos la matriz de similitud a un dataframe
idx_genres_specs = df_features_filtrados.index
df_similitud_coseno = pd.DataFrame(matriz_similitud_coseno, index=idx_genres_specs, columns=idx_genres_specs)

In [7]:
# También requerimos un dataframe para identificar los nombres de los juegos
df_id_juegos = df_steamgames[['id', 'app_name']].set_index('id')
df_id_juegos.head()

,app_name
id,
761140,Lost Summoner Kitty
643980,Ironbound
670290,Real Pool 3D - Poolians
767400,弹炸人2222
772540,Battle Royale Trainer


## Almacenamiento de Datos Procesados

In [8]:
# Almacenamos los dataframes en disco para el uso del API, cargados en 'main.py'

nombre_dir = 'data_api'
protocol_pkl = pickle.HIGHEST_PROTOCOL

# Creamos el directorio de almacén, si no existe
if not os.path.exists(nombre_dir):
    os.makedirs(nombre_dir)

# Dataframe para PlayTimeGenre
df_generos_year_playtime.to_pickle(f'./{nombre_dir}/df_generos_year_playtime.pkl', protocol=protocol_pkl)

# Dataframes para UserForGenre
df_maxplay_por_genero.to_pickle(f'./{nombre_dir}/df_maxplay_por_genero.pkl', protocol=protocol_pkl)
df_play_por_genero_por_anio.to_pickle(f'./{nombre_dir}/df_play_por_genero_por_anio.pkl', protocol=protocol_pkl)

# Dataframe para UsersRecommend
df_maxrecommend_por_anio.to_pickle(f'./{nombre_dir}/df_maxrecommend_por_anio.pkl', protocol=protocol_pkl)

# Dataframe para UsersNotRecommend
df_minrecommend_por_anio.to_pickle(f'./{nombre_dir}/df_minrecommend_por_anio.pkl', protocol=protocol_pkl)

# Dataframe para sentiment_analysis
df_sentiment_por_anio.to_pickle(f'./{nombre_dir}/df_sentiment_por_anio.pkl', protocol=protocol_pkl)

# Dataframe para sentiment_analysis
df_similitud_coseno.to_pickle(f'./{nombre_dir}/df_similitud_coseno.pkl', protocol=protocol_pkl)
df_id_juegos.to_pickle(f'./{nombre_dir}/df_id_juegos.pkl', protocol=protocol_pkl)